In [1]:
import logging
from setting.db import SessionLocal
from llm_inference.base import LLMInterface
from graph.graph_knowledge_base import GraphKnowledgeBase, SearchAction

logging.basicConfig(level=logging.INFO)

llm_client = LLMInterface("openai", "gpt-4o")

gkb = GraphKnowledgeBase(llm_client, "entities_150001", "relationships_150001", "chunks_150001")
session = SessionLocal()

In [2]:
query = "在 TiDB 中, 如果某个节点发生故障 (down机), 并且该节点的实例一直存在, 那么在故障节点的实例副本全部迁移完成后, down-peer 的数量会减少吗？请详细说明 TiDB 的副本迁移机制和 down-peer 数量变化的过程。"
model_kwargs = {
    "options": {
        "num_ctx": 8092,
        "num_gpu": 80,
        "num_predict": 10000,
        "temperature": 0.1,
    }
}
model_kwargs = {}

In [ ]:
from graph.query_analyzer import DeepUnderstandingAnalyzer

analyzer = DeepUnderstandingAnalyzer(llm_client)
analysis_res = analyzer.perform(query)
print(analysis_res)

In [4]:
action_history = []
current_findings = []
docs = {}

next_actions = [SearchAction(
    tool="retrieve_graph_data",
    query=a
) for a in analysis_res.retrieval_queries]

reasoning = analysis_res.reasoning
queries = analysis_res.retrieval_queries

In [ ]:

knowledge_retrieved = {}
for action in next_actions:
    print(action)
    if action.tool == 'retrieve_graph_data':
        data = gkb.retrieve_graph_data(session, action.query)
    elif action.tool == 'retrieve_neighbors':
        data = gkb.retrieve_neighbors(session, action.entity_ids, action.query)
    else:
        raise ValueError(f"Invalid tool: {action.tool}")

    for doc_id, doc in data.documents.items():
        if doc_id not in knowledge_retrieved:
            knowledge_retrieved[doc_id] = doc
        
        for chunk_id, chunk in doc.chunks.items():
            if chunk_id not in knowledge_retrieved[doc_id].chunks:
                knowledge_retrieved[doc_id].chunks[chunk_id] = chunk
                continue

            existing_chunk = knowledge_retrieved[doc_id].chunks[chunk_id]
            rel_dict = {r.id: r for r in existing_chunk.relationships}
            for relationship in chunk.relationships:
                rel_id = relationship.id
                if rel_id in rel_dict:
                    rel_dict[rel_id].similarity_score = max(
                        rel_dict[rel_id].similarity_score,
                        relationship.similarity_score
                    )
                else:
                    rel_dict[rel_id] = relationship.to_dict()

            knowledge_retrieved[doc_id].chunks[chunk_id].relationships = list(rel_dict.values())

action_history.append(action)

knowledge_retrieved

In [ ]:
len(knowledge_retrieved)

In [ ]:
from graph.knowledge_synthesizer import KnowledgeSynthesizer

synthesizer = KnowledgeSynthesizer(llm_client)
result = synthesizer.iterative_answer_synthesis(
    query=query,
    documents=knowledge_retrieved,
    reasoning=reasoning
)

# Access the results
final_answer = result["final_answer"]
evolution = result["evolution_history"]

In [ ]:
print(final_answer)

In [ ]:
from knowledge_extractor.synopsis_entity import process_synopsis_entity

process_synopsis_entity(session, llm_client, query, final_answer, result["relationships"])


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-AzdqHgiKuryE7va4ADjkzvTGkN85i', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "optimized_query": "How does TiDB\'s replica migration mechanism affect the number of down-peers when a node fails, and what is the process for reducing down-peer count after all replicas are migrated?"\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739254525, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_50cad350e4', usage=CompletionUsage(completion_tokens=49, prompt_tokens=169, total_tokens=218, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


930025


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...

[0.03642003983259201, 0.003959620837122202, 0.046535372734069824, 0.006016566418111324, -0.0013460895279422402, 0.037073422223329544, 0.026546699926257133, 0.028627846390008926, -0.013696839101612568, -0.04404284060001373, 0.01873030699789524, 0.003968695644289255, -0.02606271393597126, 0.008421378210186958, -0.033975906670093536, -0.010393626056611538, 0.04353465512394905, -0.030491197481751442, -0.006146638188511133, 0.07201730459928513, -0.013200752437114716, 0.02314669080078602, 0.07245289534330368, -0.0018845254089683294, -0.007798244711011648, 0.015439193695783615, 0.026546699926257133, -0.038476984947919846, 0.004873146768659353, -0.03465348854660988, 0.03482288122177124, -0.031120380386710167, -0.046002987772226334, -0.009655545465648174, 0.0303702000528574, 0.023981569334864616, 0.0007312744855880737, 0.022940995171666145, -0.011319251731038094, 0.009534548968076706, 0.026740295812487602, -0.0014217125717550516, -0.03644423931837082, 0.0330321304500103, -0.07216250151395798, 0

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[0.03642003983259201, 0.003959620837122202, 0.046535372734069824, 0.006016566418111324, -0.0013460895279422402, 0.037073422223329544, 0.026546699926257133, 0.028627846390008926, -0.013696839101612568, -0.04404284060001373, 0.01873030699789524, 0.003968695644289255, -0.02606271393597126, 0.008421378210186958, -0.033975906670093536, -0.010393626056611538, 0.04353465512394905, -0.030491197481751442, -0.006146638188511133, 0.07201730459928513, -0.013200752437114716, 0.02314669080078602, 0.07245289534330368, -0.0018845254089683294, -0.007798244711011648, 0.015439193695783615, 0.026546699926257133, -0.038476984947919846, 0.004873146768659353, -0.03465348854660988, 0.03482288122177124, -0.031120380386710167, -0.046002987772226334, -0.009655545465648174, 0.0303702000528574, 0.023981569334864616, 0.0007312744855880737, 0.022940995171666145, -0.011319251731038094, 0.009534548968076706, 0.026740295812487602, -0.0014217125717550516, -0.03644423931837082, 0.0330321304500103, -0.07216250151395798, 0

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[0.0364183634519577, 0.004125801380723715, 0.04653323069214821, 0.006116107571870089, -0.001373250619508326, 0.037120115011930466, 0.026327693834900856, 0.028723321855068207, -0.013829299248754978, -0.043895624577999115, 0.01860845275223255, 0.003959438297897577, -0.026085712015628815, 0.00846333708614111, -0.03409533202648163, -0.010495990514755249, 0.043653640896081924, -0.030489793047308922, -0.006231049075722694, 0.07196559011936188, -0.013224342837929726, 0.02327871508896351, 0.07240115851163864, -0.0018814138602465391, -0.007906777784228325, 0.015486879274249077, 0.026642272248864174, -0.038523610681295395, 0.004812426865100861, -0.03486967459321022, 0.03489387407898903, -0.031143145635724068, -0.046073466539382935, -0.009697448462247849, 0.03024781122803688, 0.02405305951833725, 0.000608737173024565, 0.02293993905186653, -0.01140342466533184, 0.009636952541768551, 0.026981046423316002, -0.0013838373124599457, -0.03629737347364426, 0.033127401024103165, -0.07225596904754639, 0.03